In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from procedures import create_binance_bot, create_binance_bot_spot
from pure_funcs import get_template_live_config, ts_to_date
from njit_funcs import round_dynamic
from time import time
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)

In [ ]:
config = get_template_live_config()
config['symbol'] = 'XMRUSDT' # dummy symbol
config['exchange'] = 'binance'
config['market_type'] = 'futures'
config['user'] = 'user_name'
try:
    await bot.session.close()
except:
    pass
if config['market_type'] == 'spot':
    bot = await create_binance_bot_spot(config)
else:
    bot = await create_binance_bot(config)

await bot.update_position()
balance = bot.position['wallet_balance']

n_days = 30
start_time = (time() - 60 * 60 * 24 * n_days) * 1000
income = await bot.get_all_income(start_time)
idf = pd.DataFrame(income)
idf.loc[:,'datetime'] = idf.timestamp.apply(ts_to_date)
idf.index = idf.timestamp
idf

In [ ]:
print(f'current balance {round_dynamic(balance, 5)}')
print(f'abs adg {round_dynamic(idf.income.sum() / n_days, 4)} {idf.iloc[-1]["token"]}')
print(f'pct adg {((balance + idf.income.sum()) / balance) ** (1 / n_days) - 1:.5f}')

idf.income.cumsum().plot()

In [ ]:
idf.tail(20)

In [ ]:
day = 1000 * 60 * 60 * 24
days = idf.timestamp // day * day
groups = idf.groupby(days)
daily_income = groups.income.sum()
cumulative = daily_income.cumsum()
starting_balance = balance - cumulative.iloc[-1]
plus_balance = cumulative + starting_balance
daily_pct_change = plus_balance / plus_balance.shift() - 1
bdf = pd.DataFrame({'abs_income': daily_income.values,
                    'gain': daily_pct_change.values,
                    'cumulative': plus_balance.values}, index=[ts_to_date(x) for x in days.unique()])
bdf

In [ ]:
bdf.gain.plot()